In [20]:
import sys
import os

sys.path += [
    '/home/ashwin/local/llama-agent-system',
    f'{os.path.expanduser("~/llama-agent-system")}'
]

# Uncomment your path in each list
MP1_CHECKPOINT_DIR = [
    # "/home/ashwin/local/checkpoints/llama3_7b_01_05_2024_torchx_launch_v5-qzn2md/",
    "/home/dalton/llama3_7b_01_05_2024_torchx_launch_v5-qzn2md"
][0]

MP1_TOKENIZER_PATH = [
    # "/home/ashwin/local/checkpoints/llama3_7b_01_05_2024_torchx_launch_v5-qzn2md/cl_toplang_128k",
    "/home/dalton/cl_toplang_128k",
][0]

MP8_CHECKPOINT_DIR = [
    os.path.abspath(os.path.expanduser("~/models/llama3_70b_full_rlhf3_14042024/"))
][0]

MP8_TOKENIZER_PATH = [
    os.path.abspath(os.path.expanduser("~/models/llama3_70b_full_rlhf3_14042024/cl_toplang_128k.model"))
][0]

os.chdir('/home/dalton/llama-agent-system/') # TODO: needed to get pwd for examples/ to work

In [2]:
from models.llama3 import (
    Attachment,
    Message,
)

from toolchain.inference import GeneratorArgs, LlamaModelParallelGenerator
from agentic_system import (
    AgenticSystem,
    BraveSearchTool,
    CodeInterpreterTool,
    PhotogenTool,
    with_safety,
    WolframAlphaTool,
)

args = GeneratorArgs(
    ckpt_dir=MP8_CHECKPOINT_DIR,
    tokenizer_path=MP8_TOKENIZER_PATH,
    mock_generation=False,
    model_parallel_size=8,
    max_seq_len=2048,
    max_batch_size=4,
)

generator = LlamaModelParallelGenerator(args)
generator.start() # Loads model into memory

> initializing model parallel with size 8
> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 9.16 seconds
NCCL version 2.20.5+cuda12.4
Finished model load YES READY
[debug] worker process done


W0618 12:21:20.390998 140511191855616 torch/distributed/elastic/multiprocessing/api.py:727] Closing process 1376314 via signal SIGTERM
W0618 12:21:20.516720 140511191855616 torch/distributed/elastic/multiprocessing/api.py:727] Closing process 1376316 via signal SIGTERM
W0618 12:21:20.517873 140511191855616 torch/distributed/elastic/multiprocessing/api.py:727] Closing process 1376317 via signal SIGTERM
W0618 12:21:20.518688 140511191855616 torch/distributed/elastic/multiprocessing/api.py:727] Closing process 1376319 via signal SIGTERM
W0618 12:21:20.519294 140511191855616 torch/distributed/elastic/multiprocessing/api.py:727] Closing process 1376320 via signal SIGTERM
W0618 12:21:20.519908 140511191855616 torch/distributed/elastic/multiprocessing/api.py:727] Closing process 1376321 via signal SIGTERM
W0618 12:21:20.520506 140511191855616 torch/distributed/elastic/multiprocessing/api.py:727] Closing process 1376322 via signal SIGTERM


In [24]:

tools = [
        WolframAlphaTool(api_key="LKRWWW-J25AKURL7T"),
        BraveSearchTool(api_key="BSAo9hsA-Gr-d0O-QF65WesoYVqKoRO"),
        CodeInterpreterTool(),
        PhotogenTool(
            api_key="1078097093361062%7CHo16EWDKl2HmOHOEa426PBXVQB8",
            model="photorealism",
            dump_dir="/tmp/photogen_dump_" + os.environ["USER"],
        ),
    ]

agent = AgenticSystem(
        generator=generator,
        tools=tools,
        max_seq_len=2048,
        input_shields=[],
        output_shields=[],
    )

async def ask_about_transcript(question):
    result = await agent.run(
        dialog=[
            Message(
                role="user",
                content=[
                    f"There is a podcast transcript located at examples/transcript_shorter.txt\n{question}",
                    Attachment(
                        filepath="examples/transcript_shorter.txt",
                        mimetype="text/txt",
                    ),
                ],
            )
        ],
        temperature=0.4,
        top_p=0.9,
        max_gen_len=2048,
    )

    print(result['generation'].content)

In [25]:
await ask_about_transcript("Summarize salient points from the podcast.")

[debug] intermediate generation(0) => 
with open("examples/transcript_shorter.txt", "r") as f:
    transcript = f.read()

print(transcript)
IPython.code_interpreter>
[Message(role='ipython', content="completed\n[stdout]\n00:00:00 - Llama 3\n\nDwarkesh Patel 00:00:00\n\nMark, welcome to the podcast.\n\nMark Zuckerberg 0:00:01\n\nThanks for having me. Big fan of your podcast.\n\nDwarkesh Patel 00:00:03\n\nThank you, that's very nice of you to say. Let's start by talking about the releases that will go out when this interview goes out. Tell me about the models and Meta AI. What’s new and exciting about them?\n\nMark Zuckerberg 00:00:15\n\nI think the main thing that most people in the world are going to see is the new version of Meta AI. The most important thing that we're doing is the upgrade to the model. We're rolling out Llama-3. We're doing it both as open source for the dev community and it is now going to be powering Meta AI. There's a lot that I'm sure we'll get into around Llama-

In [26]:
await ask_about_transcript("Was Emu mentioned?")

[debug] intermediate generation(0) => 
with open('examples/transcript_shorter.txt', 'r') as f:
    transcript = f.read()
print('Emu' in transcript)
IPython.code_interpreter>
[Message(role='ipython', content='completed\n[stdout]\nFalse\n[/stdout]', eot=True, ipython=False)]
[debug] Got EOT generation(0) => 
The word "Emu" was not me...
The word "Emu" was not mentioned in the podcast transcript.


In [27]:
await ask_about_transcript("What are the key developments in the last 6 months in the podcast's subject area?")

[debug] intermediate generation(0) => 
with open("examples/transcript_shorter.txt", "r") as f:
    transcript = f.read()

# Print the transcript
print(transcript)
IPython.code_interpreter>
[Message(role='ipython', content="completed\n[stdout]\n00:00:00 - Llama 3\n\nDwarkesh Patel 00:00:00\n\nMark, welcome to the podcast.\n\nMark Zuckerberg 0:00:01\n\nThanks for having me. Big fan of your podcast.\n\nDwarkesh Patel 00:00:03\n\nThank you, that's very nice of you to say. Let's start by talking about the releases that will go out when this interview goes out. Tell me about the models and Meta AI. What’s new and exciting about them?\n\nMark Zuckerberg 00:00:15\n\nI think the main thing that most people in the world are going to see is the new version of Meta AI. The most important thing that we're doing is the upgrade to the model. We're rolling out Llama-3. We're doing it both as open source for the dev community and it is now going to be powering Meta AI. There's a lot that I'm sure we'll

In [29]:
await ask_about_transcript("Imagine these 2 people meet again in 1 year and do another episode. What could be good follow ups to ask as a host?")

[debug] intermediate generation(0) => 
with open("examples/transcript_shorter.txt", "r") as f:
    transcript = f.read()
print(transcript)
IPython.code_interpreter>
[Message(role='ipython', content="completed\n[stdout]\n00:00:00 - Llama 3\n\nDwarkesh Patel 00:00:00\n\nMark, welcome to the podcast.\n\nMark Zuckerberg 0:00:01\n\nThanks for having me. Big fan of your podcast.\n\nDwarkesh Patel 00:00:03\n\nThank you, that's very nice of you to say. Let's start by talking about the releases that will go out when this interview goes out. Tell me about the models and Meta AI. What’s new and exciting about them?\n\nMark Zuckerberg 00:00:15\n\nI think the main thing that most people in the world are going to see is the new version of Meta AI. The most important thing that we're doing is the upgrade to the model. We're rolling out Llama-3. We're doing it both as open source for the dev community and it is now going to be powering Meta AI. There's a lot that I'm sure we'll get into around Llama-3

In [30]:
generator.stop() # Release the GPU memory after you're done